In [ ]:
# Importando Librerías
import re
import json
import time
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)


In [ ]:
# with open('test.json', 'w', encoding='utf-8') as file:
#     json.dump(jojo_data, file, indent=4, sort_keys=False)

In [ ]:
# Creando WebDriver para la busqueda con Chrome
driver = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
# Ingresando a página para obtener información de las partes de Jojo's
page_url = f'https://jojowiki.com/Category:JoJo%27s_Bizarre_Adventure_Parts'
driver.get(page_url)


## Hallando URL/Imagen cada parte 

In [ ]:
# Encontrando partes
jojo_images = driver.find_elements(
    By.XPATH, '//*[@id="mw-content-text"]/div[2]/div[1]/div/div[2]/div/div[1]/a/img')
jojo_url = driver.find_elements(
    By.XPATH, '//*[@id="mw-content-text"]/div[2]/div[1]/div/div[2]/div/div[2]/a'
)

jojo_parts = {}
for image, url in zip(jojo_images, jojo_url):

    title = url.text
    url = url.get_attribute('href')

    image = image.get_attribute('src')

    image = image.replace('/thumb', '').split('/')
    image = '/'.join(image[:-1])

    jojo_parts[title] = {"url": url}
    jojo_parts[title]["image"] = image


# with open('data/parts/jojo_parts.json', 'w', encoding='utf-8') as file:
#     json.dump(jojo_parts, file, indent=4, sort_keys=False)

jojo_parts


## Hallando Info por Parte

In [ ]:
jojo_data = jojo_parts.copy()


## Characters

In [ ]:
characters_xpath = {'Phantom Blood': ['//*[@id="mw-content-text"]/div[2]/div[14]/div/div[2]/div/div[1]/a'],
                    'Battle Tendency': ['//*[@id="mw-content-text"]/div[2]/div[15]/div/div[2]/div/div[1]/a'],
                    'Stardust Crusaders': ['//*[@id="mw-content-text"]/div[2]/div[31]/div/div[2]/div/div[1]/p/span[1]/a',
                                           '//*[@id="mw-content-text"]/div[2]/div[31]/div/div[2]/div/div[1]/a'],
                    'Diamond is Unbreakable': ['//*[@id="mw-content-text"]/div[2]/div[22]/div/div[2]/div/div[1]/p/span[1]/a',
                                               '//*[@id="mw-content-text"]/div[2]/div[22]/div/div[2]/div/div[1]/a'],
                    'Vento Aureo': ['//*[@id="mw-content-text"]/div[2]/div[27]/div/div[2]/div/div[1]/p/span[1]/a',
                                    '//*[@id="mw-content-text"]/div[2]/div[27]/div/div[2]/div/div[1]/a'],
                    'Stone Ocean': ['//*[@id="mw-content-text"]/div[2]/div[13]/div/div[2]/div/div[1]/p/span[1]/a',
                                    '//*[@id="mw-content-text"]/div[2]/div[13]/div/div[2]/div/div[1]/a'],
                    'Steel Ball Run': ['//*[@id="mw-content-text"]/div[2]/div[18]/div/div[2]/div/div[1]/p/span[1]/a',
                                       '//*[@id="mw-content-text"]/div[2]/div[18]/div/div[2]/div/div[1]/a'],
                    'JoJolion': ['//*[@id="mw-content-text"]/div[2]/div[27]/div/div[2]/div/div[1]/p/span[1]/a',
                                 '//*[@id="mw-content-text"]/div[2]/div[27]/div/div[2]/div[8]/div[1]/a']}


In [ ]:
for part in jojo_data.keys():
    driver.get(jojo_data[part]["url"])

    fields = driver.find_elements(
        By.XPATH, '//*[@id="mw-content-text"]/div[2]/aside/div/h3')

    datum = driver.find_elements(
        By.XPATH, '//*[@id="mw-content-text"]/div/aside/div/div')

    for field, data in zip(fields, datum):
        resume = ' '.join([driver.find_element(
            By.XPATH, f'//*[@id="mw-content-text"]/div[2]/p[{i}]').text for i in range(3, 6)])

        resume = re.sub("\[.*?\]", "", resume)
        jojo_data[part]["resume"] = resume
        jojo_data[part][field.text.lower()] = data.text

    print(part.upper(), '\t', jojo_data[part]["url"])
    print('-'*45)

    xpath_elements = characters_xpath.get(part, None)

    temp = {}
    for xpath in xpath_elements:
        characters = driver.find_elements(By.XPATH, xpath)

        for character in characters:
            name = character.get_attribute('title')
            url = character.get_attribute('href')
            print(name, url)

            temp[name] = {"url": url}
    jojo_data[part]["characters"] = temp

    print('#'*45)


In [ ]:
jojo_data

## Get HTML Source

In [ ]:
from pathlib import Path

for part in jojo_parts.keys():
    print(part.upper())

    path = f"data\html\{part}"
    Path(path).mkdir(parents=True, exist_ok=True)

    for character in jojo_data[part]['characters'].keys():
        driver.get(jojo_data[part]['characters'][character]['url'])

        source_code = driver.page_source

        with open(f'{path}\{character}.html', 'w', encoding="utf-8") as f:
            f.write(source_code)
        f.close()

        print(part, f'{path}\{character}.html')
    
    print('-'*65)


### Characters Features

In [ ]:
with open("data/jojo_data.json") as json_file:
    jojo_data = json.load(json_file)

In [ ]:
jojo_data

In [ ]:
#tabber-62e2fd7410acef19b692bdffc36ace61 > div:nth-child(2) > p > a

In [ ]:
driver.get(jojo_data['Phantom Blood']['characters']['Jonathan Joestar']['url'])


In [ ]:
image = driver.find_element(
            By.XPATH, '/html/body/div[1]/div/div[3]/div[3]/div[5]/div[2]/aside/div[1]/div/div/div[1]/p/a/img')

In [ ]:
image = image.get_attribute('src')

image = image.replace('/thumb', '').split('/')
image = '/'.join(image[:-1])

image

In [ ]:
features = driver.find_elements(By.CLASS_NAME, "pi-item pi-data pi-item-spacing pi-border-color")
features

In [ ]:
images = driver.find_elements_by_css_selector('[id^="tabber-"]')
images

In [ ]:
images[1].text

/html/body/div[1]/div/div[3]/div[3]/div[5]/div[2]/aside/div[1]/div/div/div[1]/p/a/img
/html/body/div[1]/div/div[3]/div[3]/div[5]/div[2]/aside/div[1]/div/div/div[1]/div/div/div[1]/p/a/img

In [1]:
# Importando Librerías
import re
import json
import time
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
# Creando WebDriver para la busqueda con Chrome
driver = webdriver.Chrome(ChromeDriverManager().install())

In [2]:
with open("data/jojo_data.json") as json_file:
    jojo_data = json.load(json_file)

In [3]:
image_xpaths = ["/html/body/div[1]/div/div[3]/div[3]/div[5]/div[2]/aside/div[1]/div/div/div[1]/div/div/div[1]/p/a/img",
                "/html/body/div[1]/div/div[3]/div[3]/div[5]/div[2]/aside/div[1]/div/div/div[1]/p/a/img",
                "/html/body/div[1]/div/div[3]/div[3]/div[5]/div[2]/aside/div[1]/div/a/img"]

for part in jojo_data.keys():

    print('-'*75)
    print(part.upper())

    for character in list(jojo_data[part]['characters'].keys()):

        print(character, end=' ')
        driver.get(jojo_data[part]['characters'][character]['url'])

        # Finding image by character
        image = None
        count=0
        while image is None:
            try:
                image = driver.find_element(
                    By.XPATH, image_xpaths[count])
                image = image.get_attribute('src')                

                if '400px' in image:
                    image = image.replace('/thumb', '').split('/')
                    image = '/'.join(image[:-1])

                jojo_data[part]['characters'][character]['img'] = image
                count=0

            except:
                count+=1
                if count>len(image_xpaths):
                    break
        
        # Finding resume by character
        resume = ' '.join([driver.find_element(
            By.XPATH, f'//*[@id="mw-content-text"]/div[2]/p[{i}]').text for i in range(3, 6)])

        resume = re.sub("\[.*?\]", "", resume)
        jojo_data[part]['characters'][character]["resume"] = resume
        print(image)

---------------------------------------------------------------------------
PHANTOM BLOOD
Jonathan Joestar https://static.jojowiki.com/images/b/bd/latest/20210225235345/Jonathan_Infobox_Manga.png
Will Anthonio Zeppeli https://static.jojowiki.com/images/9/9c/latest/20210227224405/Will_Zeppeli_Infobox_Manga.png
Robert E. O. Speedwagon https://static.jojowiki.com/images/b/bc/latest/20210311114212/SPW_PB_Infobox_Manga.png
Dio Brando https://static.jojowiki.com/images/5/5f/latest/20210529185004/Dio_PB_Infobox_Manga.png
Erina Pendleton https://static.jojowiki.com/images/5/5f/latest/20210228130131/Erina_PB_Infobox_Manga.png
George Joestar I https://static.jojowiki.com/images/1/19/latest/20210228132632/George_Infobox_Manga.png
Poco https://static.jojowiki.com/images/0/0e/latest/20210228131535/Poco_Infobox_Manga.PNG
Poco's Sister https://static.jojowiki.com/images/e/e5/latest/20200110170343/Poco_Sister_Infobox_Manga.png
Tonpetty https://static.jojowiki.com/images/6/69/latest/20210228133418/Tonp

In [4]:
with open('data/jojo_data.json', 'w', encoding='utf-8') as file:
    json.dump(jojo_data, file, indent=4, sort_keys=False)

In [ ]:
jojo_data

In [ ]:
# with open('data/jojo_data.json', 'w', encoding='utf-8') as file:
#     json.dump(jojo_data, file, indent=4, sort_keys=False)

In [ ]:
jojo_data